In [1]:
import os
import time

##  2+ GB File reading 

In [2]:
os.path.getsize('C:/Jupyter/used_cars_data (2).csv')

9980208148

### Reading the data with dask

In [4]:
from dask import dataframe as dd
start = time.time()
dask_df1=dd.read_csv('C:/Jupyter/used_cars_data (2).csv')
end = time.time()
print("Read Dask:",(end-start),"sec")

Read Dask: 0.06819939613342285 sec


### Reading the data with Pandas

In [7]:
import pandas as pd
start = time.time()
df1=pd.read_csv("C:/Jupyter/used_cars_data (2).csv")
end = time.time()
print("Read Pandas:",(end-start),"sec")

MemoryError: Unable to allocate 64.0 KiB for an array with shape (8192,) and data type int64

### Reading the data with Modin and Ray

In [ ]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df=pd.read_csv("C:/Jupyter/used_cars_data (2).csv")
end = time.time()
print("Read modin and ray:",(end-start),"sec")

2022-10-05 20:52:27,906	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


## Dask has highest time efficiency for reading data above 2+ GB. Pandas, Modin and Ray have constranits.

## File Reading  <2GB

In [2]:
os.path.getsize("C:\Jupyter\insurance.csv")

54289

### Reading the data with Dask

In [3]:
from dask import dataframe as dd
start = time.time()
dask_df=dd.read_csv("C:\Jupyter\insurance.csv")
end = time.time()
print("Read Dask:",(end-start),"sec")

Read Dask: 0.029916763305664062 sec


### Reading the data with Pandas

In [5]:
import pandas as pd
start = time.time()
df=pd.read_csv("C:\Jupyter\insurance.csv")
end = time.time()
print("Read Pandas:",(end-start),"sec")

Read Pandas: 0.010935306549072266 sec


### Reading the data with Modin and Ray

In [6]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df=pd.read_csv("C:\Jupyter\insurance.csv")
end = time.time()
print("Read modin and ray:",(end-start),"sec")

2022-09-30 19:40:53,437	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8267


Read modin and ray: 2.0362818241119385 sec


## Pandas has highest time efficiency in case of smaller file size

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("C:\Jupyter\insurance.csv",delimiter=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [9]:
#No of rows
len(df.index)

1338

In [10]:
# No of columns
len(df.columns)

7

In [11]:
# removing special charecter
df.columns=df.columns.str.replace('[#,@,&]','')

In [12]:
# removing white spaces
df.columns=df.columns.str.replace(' ','')

In [13]:
data=df.columns
data

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

### Schema Validation

In [14]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re

In [109]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replace(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [42]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: insurance
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
      - age
      - sex
      - bmi
      - children
      - smoker
      - region
      - charges

Overwriting store.yaml


In [43]:
# Reading config file
import utility as util
import yaml


In [44]:
config_data = yaml.safe_load(open('store.yaml'))

In [45]:
#data of config file
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'insurance',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']}

In [48]:
# Reading process of the file using Dask
import pandas as pd
df_sample = pd.read_csv('C:\Jupyter\insurance.csv',delimiter=',')
df_sample.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [49]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = "C:/Jupyter/"+ config_data['file_name'] + f'.{file_type}'

In [50]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [55]:

print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])


columns of files are: Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')
columns of YAML are: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']


In [100]:
import datetime
import csv

import pandas as pd
df = pd.read_csv('C:\Jupyter\insurance.csv',delimiter=',')


In [104]:
read_file = pd.read_csv (r'C:\Jupyter\insurance.csv',delimiter=',')
read_file.to_csv (r'C:\Users\innam\Desktop\insurance.csv',sep='|', index=None)

In [105]:
os.path.getsize(r'C:\Users\innam\Desktop\insurance.csv')

55690

In [106]:
df = pd.read_csv(r'C:\Users\innam\Desktop\insurance.csv',delimiter='|')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB
